# Multicriterio


In [7]:
import numpy as np
import pandas as pd

xls=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\imdbmoviesdata.xlsx")
datos=pd.read_excel(xls,"tmdb_movies_data")
tabla_AHP=pd.read_excel(xls,"AHP")
tabla_AHP_gen=pd.read_excel(xls,"AHP_genero")
datos.head()
tabla_AHP.head()
#[65,90,120,180,340]


,Unnamed: 0,Popularidad,Presupuesto,Ventas,Duración,Género,Votos,PromedioVotos,Estreno
0,Popularidad,1.000000,2.000000,3.0,6.0,0.3,0.8,0.5,1
1,Presupuesto,0.500000,1.000000,0.8,0.9,0.2,0.9,0.5,2
2,Ventas,0.333333,1.250000,1.0,2.0,0.5,0.8,0.7,2
3,Duración,0.166667,1.111111,0.5,1.0,0.1,0.7,0.6,3
4,Género,3.333333,5.000000,2.0,10.0,1.0,5.0,4.0,7


In [33]:
def calc_ponder(matr:np.array,tol:float):
    """Recibe:
    matr: matriz nxn de comparación AHP dos a dos recíproca.
    tol: tolerancia de variación entre vectores suma normalizados.
    Retorna:
    vect_pond: vector ponderación. Eleva error si no se cumple consistencia"""
    n,_=matr.shape
    epsilon=np.array([tol+1])
    i=2
    prev_v_norm=matr.sum(axis=1)/matr.sum()
    while (epsilon>=tol).any():
        if i>6:
            raise Exception("Matriz no consistente")
        A_n=np.linalg.matrix_power(matr,i)#Eleva error si no es cuadrada
        v_norm=A_n.sum(axis=1)/A_n.sum()
        epsilon=max(v_norm-prev_v_norm)
        prev_v_norm=v_norm
    
    v1=np.lin
    
    indice_RI={1:0,3:0.58,4:0.89,5:1.11,6:1.24,7:1.32,8:1.4,9:1.5,10:1.49}
    RI=indice_RI[n]


#datos["Género"].unique()
matr=np.array([[1,3,5,3,5],[0.33,1,2,1,3],[0.2,0.5,1,1,2],[0.33,1,1,1,4],[0.2,0.33,0.5,0.25,1]])
calc_ponder(matr,2)

0
2
[0.47367703 0.18037076 0.11686306 0.16841521 0.06067394]


In [22]:
b=np.array([[3,1,3]])
while b<3:
    print("etf")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [5]:
tabla_AHP_gen.head()

,Unnamed: 0,Action,Adventure,Animation,Comedy,Crime,Drama,Familiy,Fantasy,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,Action,1.000000,2.000000,1.500000,3.0,4.0,1.5,4.0,3.0,5.0,2.0,3.0,1.5,2.0,1.5,1.5,2.0
1,Adventure,0.500000,1.000000,0.900000,1.5,2.0,1.0,2.0,1.0,3.0,1.0,2.0,0.5,1.0,1.5,0.8,1.0
2,Animation,0.666667,1.111111,1.000000,2.0,3.0,1.0,3.0,2.0,4.0,1.5,2.0,1.0,1.5,1.0,1.0,1.5
3,Comedy,0.333333,0.666667,0.500000,1.0,2.0,0.5,3.0,1.0,3.0,0.5,1.0,0.5,0.8,0.5,0.5,0.8
4,Crime,0.250000,0.500000,0.333333,0.5,1.0,0.3,2.0,0.8,2.0,0.3,0.5,0.4,0.5,0.3,0.3,0.5
